In [ ]:
import threading

from flask import Flask, request, render_template_string, Response, send_from_directory, abort
import os
import html

# Disable default Flask static folder because we serve static files manually
app = Flask(__name__, static_folder=None)

# Base directory of project folder
BASE_DIR = os.path.dirname(os.path.abspath(__file__))


# ---------------------------------------------------------------
# Load HTML template (Task requirement: with open + try/except)
# ---------------------------------------------------------------
def load_template(path='index.html'):
    """
    Reads and returns HTML file content.
    Handles errors using try/except for better debugging.
    """
    full_path = os.path.join(BASE_DIR, path)
    try:
        with open(full_path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        return f"<h2>Template not found: {html.escape(path)}</h2>"
    except Exception as e:
        return f"<h2>Error loading template: {html.escape(str(e))}</h2>"


# ---------------------------------------------------------------
# Home Route - Serve index.html
# ---------------------------------------------------------------
@app.route("/", methods=["GET"])
def home():
    content = load_template("index.html")
    return Response(content, mimetype="text/html")


# ---------------------------------------------------------------
# Static Files Route (CSS, JS)
# ---------------------------------------------------------------
@app.route("/<path:filename>")
def serve_static(filename):
    # Allow only specific files
    allowed = {"styles.css", "login.js", "favicon.ico"}

    if filename not in allowed:
        abort(404)

    try:
        return send_from_directory(BASE_DIR, filename)
    except FileNotFoundError:
        abort(404)
    except Exception as e:
        return (
            Response(
                f"<h2>Error serving file {html.escape(filename)}: {html.escape(str(e))}</h2>",
                mimetype="text/html",
            ),
            500,
        )


# ---------------------------------------------------------------
# Login Route (Task 3 + Server-side validation)
# ---------------------------------------------------------------
@app.route("/login", methods=["POST"])
def login():
    username = request.form.get("username", "").strip()
    password = request.form.get("password", "").strip()

    # Server-side validation
    if not username or not password:
        return (
            """
            <h3>Login failed: username and password required.</h3>
            <p><a href="/">Back to home</a></p>
            """,
            400,
        )

    # Print the username to console (Task requirement)
    print(f"Student logged in: {username}")

    safe_name = html.escape(username)

    return f"""
        <h2>Welcome, {safe_name}!</h2>
        <p>This response was served by Flask.</p>
        <p><a href="/">Back to home</a></p>
    """


# ---------------------------------------------------------------
# Run the app (for local testing)
# ---------------------------------------------------------------
if __name__ == "__main__":
    app.run(debug=True, host="127.0.0.1", port=int(os.environ.get("PORT", 5000)))
